![image_2.png](./img/image_2.png)





## Summary

* #### [Introducción](#introduccion)
* #### [Hipotesis](#hipotesis)
* #### [Procedimiento](#procedimiento)
    * #### [Extracción](#extraccion)
    * #### [Limpieza de datos (paso 1)](#limpieza)
    * #### [Unificación de datos](#unificacion)
    * #### [Limpieza de datos (paso 2)](#limpieza2)
* #### [Inicio del análisis](#analisis)

# **Análisis exploratorio de Datos (EDA)**

### Introduccion <a id="introduccion"></a>

## México y la migración ¿Cuáles son sus causas?

¿Depende de la causa de migración la razón por la cuál un mexicano migra a los Estados Unidos Américanos o a otros países? 

**Hipótesis a contrastar**: Se asume que la causa de migración determina las razones por la cuales los mexicanos migran a los Estados Unidos Américanos u a otros países. <a id="hipotesis"></a>









### Procedimiento <a id="procedimiento"></a>

Como primer paso se importan las librerias que se necesitarán para poder realizar el anális exploratorio de datos.

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import plotly.express as px
import os.path 
import sqlite3

En esta sección leeremos los archivos que necesitaremos para hacer tal análisis

>En esta parte nos enfrentamoss a dificultades para poder acceder al archivo puesto que nos marca un error diciendo que el archivo no se puede encontrar, 
por lo tanto se ha revisado en stack overflow y encontrado una solución que ha funcionado con agreagr la letra r al incio
>> El siguiente reto al que nos encontramos es que el archivo en excel esta bloqueado, así que no podemos ajustar desde el excel directamente, por lo que tendrémos que renombrar las columnas usando métodos de pandas

### Extracción de datos <a id="extraccion"></a>

In [58]:

df_INEGI_Yr2020_07 = pd.read_excel(r"/Users/stefa/REPO_PRUEBA/EDA_Proyecto/src/data/cpv2020.xlsx",sheet_name="07",skiprows=[0,1,2,3,4,5]) #se utiliza letra r para leer el archivo
df_INEGI_Yr2020_07.head(5)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,Por inseguridad delictiva o violencia,Por desastres naturales,Lo deportaron,Otra causa,No especificado
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Estados Unidos MexicanosTotal,Estados Unidos Mexicanos,Total,Total,6888490.0,1129177.0,855435.0,2515611.0,636148.0,463170.0,274158.0,24742.0,41390.0,860165.0,88494.0
2,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,24816.0,1760.0,587.0,67922.0,7661.0
3,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,542634.0,6758.0,7067.0,385222.0,1102.0,58274.0,26181.0,1826.0,630.0,50036.0,5538.0
4,Estados Unidos Mexicanos15-19 años,Estados Unidos Mexicanos,Total,15-19 años,595630.0,73319.0,22189.0,270949.0,41917.0,117612.0,23439.0,1543.0,482.0,37946.0,6234.0


>En esta parte nos encontramos un nuevo reto ya que las filas con los títulos estaban concatenadas, así que parecía que necesitaba poner un salto de carro, al final
buscando en las librerías y Stack overflow he visto que con el inplace funcionaba el cambio de nombre

In [59]:
df_INEGI_Yr2020_07.rename(columns ={"Unnamed: 0": "Unicode", "Unnamed: 1":"Entidad Federativa de residencia actual","Unnamed: 2":"Género","Unnamed: 3": "Grupos quinquenales de edad","Unnamed: 4" : "Población de 5 años y más migrante"},inplace=True)
df_INEGI_Yr2020_07.head(5) #Cuando se imprime se puede notar que las columnas están renombradas

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,Por inseguridad delictiva o violencia,Por desastres naturales,Lo deportaron,Otra causa,No especificado
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Estados Unidos MexicanosTotal,Estados Unidos Mexicanos,Total,Total,6888490.0,1129177.0,855435.0,2515611.0,636148.0,463170.0,274158.0,24742.0,41390.0,860165.0,88494.0
2,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,24816.0,1760.0,587.0,67922.0,7661.0
3,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,542634.0,6758.0,7067.0,385222.0,1102.0,58274.0,26181.0,1826.0,630.0,50036.0,5538.0
4,Estados Unidos Mexicanos15-19 años,Estados Unidos Mexicanos,Total,15-19 años,595630.0,73319.0,22189.0,270949.0,41917.0,117612.0,23439.0,1543.0,482.0,37946.0,6234.0


>El siguiente paso es traer el Sheet número 5, el cual tiene los datos de lugares a los que los mexicanos han migrado.

In [60]:
df_INEGI_Yr2020_5 = pd.read_excel(r"/Users/stefa/REPO_PRUEBA/EDA_Proyecto/src/data/cpv2020.xlsx",sheet_name="05",skiprows=[0,1,2,3,4,5],usecols="A:E,J:N") #Se observa que se traen las columnas que únicamente necesitamos para nuestro análisis
df_INEGI_Yr2020_5.head(5)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,En otra entidad o país,Unnamed: 10,Unnamed: 11,Unnamed: 12,No especificado
0,NaN,NaN,NaN,NaN,NaN,Total,En otra entidad,En los Estados Unidos de América,En otro país,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Estados Unidos MexicanosTotal,Estados Unidos Mexicanos,Total,Total,115693273.0,4357929,3807844,374155,175930,259750.0
3,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,10764379.0,385509,353149,20662,11698,13876.0
4,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,10943540.0,334988,304784,20721,9483,8895.0


>En este apartado se renombran las columnas puesto que el Excel esta bloqueado y no se pueden modificar manualmente en el excel, por lo cual se ha tenido que utilizar las herramientas de pandas para nombrar las columnas como en la tabla anterior

In [61]:

df_INEGI_Yr2020_5.rename(columns ={"Unnamed: 0":"Unicode","Unnamed: 1":"Entidad Federativa de residencia actual","Unnamed: 2":"Género","Unnamed: 3": "Grupos quinquenales de edad","Unnamed: 4" : "Población de 5 años y más migrante","En otra entidad del país":"Total en otra entidad del país","Unnamed: 10":"En otra entidad","Unnamed: 11":"En los Estados Unidos de América","Unnamed: 12":"En otro país"},inplace=True)
df_INEGI_Yr2020_5 #se renombran las columnas utilizando pandas

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado
0,NaN,NaN,NaN,NaN,NaN,Total,En otra entidad,En los Estados Unidos de América,En otro país,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Estados Unidos MexicanosTotal,Estados Unidos Mexicanos,Total,Total,115693273.0,4357929,3807844,374155,175930,259750.0
3,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,10764379.0,385509,353149,20662,11698,13876.0
4,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,10943540.0,334988,304784,20721,9483,8895.0
...,...,...,...,...,...,...,...,...,...,...
1779,32 Zacatecas65-69 años,32 Zacatecas,Mujeres,65-69 años,23212.0,387,224,162,1,11.0
1780,32 Zacatecas70-74 años,32 Zacatecas,Mujeres,70-74 años,18020.0,214,130,84,0,12.0
1781,32 Zacatecas75-79 años,32 Zacatecas,Mujeres,75-79 años,13342.0,160,97,63,0,5.0
1782,32 Zacatecas80-84 años,32 Zacatecas,Mujeres,80-84 años,9417.0,85,57,28,0,5.0


### Limpieza de datos (paso 1)  <a id="limpieza"></a>

#### Tabla 07


In [62]:
df_INEGI_Yr2020_07.info() #aunque pareciera que no hay nulos, en la tabla de abajo podemos ver que hay líneas enteras que contienen Nulos, las cuales hay que eliminar

df_INEGI_Yr2020_07.loc[df_INEGI_Yr2020_07 ["Entidad Federativa de residencia actual"].isna()] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unicode                                  1782 non-null   object 
 1   Entidad Federativa de residencia actual  1783 non-null   object 
 2   Género                                   1782 non-null   object 
 3   Grupos quinquenales de edad              1782 non-null   object 
 4   Población de 5 años y más migrante       1782 non-null   float64
 5   Buscar trabajo                           1782 non-null   float64
 6   Cambio u oferta de trabajo               1782 non-null   float64
 7   Reunirse con la familia                  1782 non-null   float64
 8   Se casó o unió                           1782 non-null   float64
 9   Estudiar                                 1782 non-null   float64
 10  Por inseguridad delictiva o violencia    1782 no

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,Por inseguridad delictiva o violencia,Por desastres naturales,Lo deportaron,Otra causa,No especificado
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
df_INEGI_Yr2020_07.drop([0],inplace=True) #Se elimina la primera fila de NaN a través de Index 0
df_INEGI_Yr2020_07.dropna(inplace=True) #Se eliminan los NaN de otras filas
df_INEGI_Yr2020_07.head(5)


,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,Por inseguridad delictiva o violencia,Por desastres naturales,Lo deportaron,Otra causa,No especificado
1,Estados Unidos MexicanosTotal,Estados Unidos Mexicanos,Total,Total,6888490.0,1129177.0,855435.0,2515611.0,636148.0,463170.0,274158.0,24742.0,41390.0,860165.0,88494.0
2,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,24816.0,1760.0,587.0,67922.0,7661.0
3,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,542634.0,6758.0,7067.0,385222.0,1102.0,58274.0,26181.0,1826.0,630.0,50036.0,5538.0
4,Estados Unidos Mexicanos15-19 años,Estados Unidos Mexicanos,Total,15-19 años,595630.0,73319.0,22189.0,270949.0,41917.0,117612.0,23439.0,1543.0,482.0,37946.0,6234.0
5,Estados Unidos Mexicanos20-24 años,Estados Unidos Mexicanos,Total,20-24 años,864222.0,208504.0,73593.0,203711.0,130552.0,164553.0,20802.0,1394.0,1704.0,49431.0,9978.0


In [64]:
df_INEGI_Yr2020_07.loc[df_INEGI_Yr2020_07 ["Entidad Federativa de residencia actual"].isna()] #Corroboramos que no haya más Nulos


,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,Por inseguridad delictiva o violencia,Por desastres naturales,Lo deportaron,Otra causa,No especificado


In [65]:
mask_drop_GQuin_07 = df_INEGI_Yr2020_07['Grupos quinquenales de edad'] == 'Total' 
'''Se elimina el Total de Grupos quinquenales de edad utlizando una máscara ya que el total lo podemos calcular utilizando python
'''
df_INEGI_Yr2020_07 = df_INEGI_Yr2020_07 [~mask_drop_GQuin_07]
df_INEGI_Yr2020_07.head(5)

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,Por inseguridad delictiva o violencia,Por desastres naturales,Lo deportaron,Otra causa,No especificado
2,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,24816.0,1760.0,587.0,67922.0,7661.0
3,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,542634.0,6758.0,7067.0,385222.0,1102.0,58274.0,26181.0,1826.0,630.0,50036.0,5538.0
4,Estados Unidos Mexicanos15-19 años,Estados Unidos Mexicanos,Total,15-19 años,595630.0,73319.0,22189.0,270949.0,41917.0,117612.0,23439.0,1543.0,482.0,37946.0,6234.0
5,Estados Unidos Mexicanos20-24 años,Estados Unidos Mexicanos,Total,20-24 años,864222.0,208504.0,73593.0,203711.0,130552.0,164553.0,20802.0,1394.0,1704.0,49431.0,9978.0
6,Estados Unidos Mexicanos25-29 años,Estados Unidos Mexicanos,Total,25-29 años,954730.0,243836.0,159400.0,215073.0,161690.0,47077.0,25043.0,1767.0,4068.0,85445.0,11331.0


#### Tabla 05


In [66]:
df_INEGI_Yr2020_5.info() #aunque pareciera que no hay nulos, en la tabla de abajo podemos ver que hay líneas enteras que contienen Nulos, las cuales hay que eliminar

df_INEGI_Yr2020_5.loc[df_INEGI_Yr2020_5 ["Entidad Federativa de residencia actual"].isna()] 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1784 entries, 0 to 1783
Data columns (total 10 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unicode                                  1782 non-null   object 
 1   Entidad Federativa de residencia actual  1782 non-null   object 
 2   Género                                   1782 non-null   object 
 3   Grupos quinquenales de edad              1782 non-null   object 
 4   Población de 5 años y más migrante       1782 non-null   float64
 5   En otra entidad o país                   1783 non-null   object 
 6   En otra entidad                          1783 non-null   object 
 7   En los Estados Unidos de América         1783 non-null   object 
 8   En otro país                             1783 non-null   object 
 9   No especificado                          1782 non-null   float64
dtypes: float64(2), object(8)
memory usage: 139.5+ KB

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado
0,NaN,NaN,NaN,NaN,NaN,Total,En otra entidad,En los Estados Unidos de América,En otro país,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df_INEGI_Yr2020_5.drop([0],inplace=True) #Se elimina la primera fila de NaN a través de Index 0
df_INEGI_Yr2020_5.dropna(inplace=True) #Se eliminan los NaN de otras filas
df_INEGI_Yr2020_5.head(5)

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado
2,Estados Unidos MexicanosTotal,Estados Unidos Mexicanos,Total,Total,115693273.0,4357929,3807844,374155,175930,259750.0
3,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,10764379.0,385509,353149,20662,11698,13876.0
4,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,10943540.0,334988,304784,20721,9483,8895.0
5,Estados Unidos Mexicanos15-19 años,Estados Unidos Mexicanos,Total,15-19 años,10806690.0,371644,345520,15908,10216,10996.0
6,Estados Unidos Mexicanos20-24 años,Estados Unidos Mexicanos,Total,20-24 años,10422095.0,553636,515834,19721,18081,26457.0


In [68]:
df_INEGI_Yr2020_5.loc[df_INEGI_Yr2020_5 ["Entidad Federativa de residencia actual"].isna()] #Corroboramos que no haya más Nulos

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado


In [70]:
mask_drop_GQuin_5 = df_INEGI_Yr2020_5['Grupos quinquenales de edad'] == 'Total' 
'''Se elimina el Total de Grupos quinquenales de edad utlizando una máscara ya que el total lo podemos calcular utilizando python
'''
df_INEGI_Yr2020_5 = df_INEGI_Yr2020_5 [~mask_drop_GQuin_5]
df_INEGI_Yr2020_5.head(5)

,Unicode,Entidad Federativa de residencia actual,Género,Grupos quinquenales de edad,Población de 5 años y más migrante,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado
3,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,10764379.0,385509,353149,20662,11698,13876.0
4,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,10943540.0,334988,304784,20721,9483,8895.0
5,Estados Unidos Mexicanos15-19 años,Estados Unidos Mexicanos,Total,15-19 años,10806690.0,371644,345520,15908,10216,10996.0
6,Estados Unidos Mexicanos20-24 años,Estados Unidos Mexicanos,Total,20-24 años,10422095.0,553636,515834,19721,18081,26457.0
7,Estados Unidos Mexicanos25-29 años,Estados Unidos Mexicanos,Total,25-29 años,9993001.0,604620,543923,34002,26695,38010.0


### Unificación de valores de las tablas <a id="unificacion"></a>

*En esta sección hacemos la combinación de las tablas 07 y 5. Sin embargo, podemos ver que se han creados una **segunda limpieza** tendrá que llevarse acabo*

> Antes de continuar con la limpieza se ha observado que las filas han incrementadp de 1683 en cada tabla a 85833 por lo que procedemos a verificar
la solución que se encontró fue crear una nueva columna llamada "Unicode" la cual ayuda a unificar mediante un único valor que tenga como referencia
la *Entidad Federativa de residencia actual* y los *Grupos quinquenales de edad* reduciendo las líneas a 5049 como se aprecia en la tabla de abajo.

In [73]:
df_INEGI_merged = df_INEGI_Yr2020_07.merge(df_INEGI_Yr2020_5, how='left',on="Unicode") #Combinación de tablas 07 y 5
df_INEGI_merged


,Unicode,Entidad Federativa de residencia actual_x,Género_x,Grupos quinquenales de edad_x,Población de 5 años y más migrante_x,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,...,No especificado_x,Entidad Federativa de residencia actual_y,Género_y,Grupos quinquenales de edad_y,Población de 5 años y más migrante_y,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado_y
0,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,...,7661.0,Estados Unidos Mexicanos,Total,05-09 años,10764379.0,385509,353149,20662,11698,13876.0
1,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,...,7661.0,Estados Unidos Mexicanos,Hombres,05-09 años,5453091.0,194822,178194,10544,6084,7025.0
2,Estados Unidos Mexicanos05-09 años,Estados Unidos Mexicanos,Total,05-09 años,624339.0,0.0,0.0,479214.0,0.0,42379.0,...,7661.0,Estados Unidos Mexicanos,Mujeres,05-09 años,5311288.0,190687,174955,10118,5614,6851.0
3,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,542634.0,6758.0,7067.0,385222.0,1102.0,58274.0,...,5538.0,Estados Unidos Mexicanos,Total,10-14 años,10943540.0,334988,304784,20721,9483,8895.0
4,Estados Unidos Mexicanos10-14 años,Estados Unidos Mexicanos,Total,10-14 años,542634.0,6758.0,7067.0,385222.0,1102.0,58274.0,...,5538.0,Estados Unidos Mexicanos,Hombres,10-14 años,5554260.0,169188,154091,10249,4848,4571.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,32 Zacatecas80-84 años,32 Zacatecas,Mujeres,80-84 años,149.0,2.0,6.0,97.0,0.0,0.0,...,2.0,32 Zacatecas,Hombres,80-84 años,8641.0,95,49,46,0,4.0
5045,32 Zacatecas80-84 años,32 Zacatecas,Mujeres,80-84 años,149.0,2.0,6.0,97.0,0.0,0.0,...,2.0,32 Zacatecas,Mujeres,80-84 años,9417.0,85,57,28,0,5.0
5046,32 Zacatecas85 años y más,32 Zacatecas,Mujeres,85 años y más,193.0,1.0,2.0,129.0,0.0,0.0,...,3.0,32 Zacatecas,Total,85 años y más,17506.0,201,129,71,1,4.0
5047,32 Zacatecas85 años y más,32 Zacatecas,Mujeres,85 años y más,193.0,1.0,2.0,129.0,0.0,0.0,...,3.0,32 Zacatecas,Hombres,85 años y más,8233.0,83,46,37,0,2.0


### Limpieza de datos (paso 2)  <a id="limpieza2"></a>

#### Tabla merged

> En este apartado verificamos que no haya NaN 

In [80]:
df_INEGI_merged.columns

Index(['Unicode', 'Entidad Federativa de residencia actual_x', 'Género_x',
       'Grupos quinquenales de edad_x', 'Población de 5 años y más migrante_x',
       'Buscar trabajo', 'Cambio u oferta de trabajo',
       'Reunirse con la familia', 'Se casó o unió', 'Estudiar',
       'Por inseguridad delictiva o violencia', 'Por desastres naturales',
       'Lo deportaron', 'Otra causa', 'No especificado_x',
       'Entidad Federativa de residencia actual_y', 'Género_y',
       'Grupos quinquenales de edad_y', 'Población de 5 años y más migrante_y',
       'En otra entidad o país', 'En otra entidad',
       'En los Estados Unidos de América', 'En otro país',
       'No especificado_y'],
      dtype='object')

In [85]:
df_INEGI_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5049 entries, 0 to 5048
Data columns (total 24 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unicode                                    5049 non-null   object 
 1   Entidad Federativa de residencia actual_x  5049 non-null   object 
 2   Género_x                                   5049 non-null   object 
 3   Grupos quinquenales de edad_x              5049 non-null   object 
 4   Población de 5 años y más migrante_x       5049 non-null   float64
 5   Buscar trabajo                             5049 non-null   float64
 6   Cambio u oferta de trabajo                 5049 non-null   float64
 7   Reunirse con la familia                    5049 non-null   float64
 8   Se casó o unió                             5049 non-null   float64
 9   Estudiar                                   5049 non-null   float64
 10  Por inseguridad delictiv

In [88]:
df_INEGI_merged.dropna(inplace=True)
df_INEGI_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5049 entries, 0 to 5048
Data columns (total 24 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unicode                                    5049 non-null   object 
 1   Entidad Federativa de residencia actual_x  5049 non-null   object 
 2   Género_x                                   5049 non-null   object 
 3   Grupos quinquenales de edad_x              5049 non-null   object 
 4   Población de 5 años y más migrante_x       5049 non-null   float64
 5   Buscar trabajo                             5049 non-null   float64
 6   Cambio u oferta de trabajo                 5049 non-null   float64
 7   Reunirse con la familia                    5049 non-null   float64
 8   Se casó o unió                             5049 non-null   float64
 9   Estudiar                                   5049 non-null   float64
 10  Por inseguridad delictiv

In [83]:
df_INEGI_merged.loc[df_INEGI_merged["Buscar trabajo"].isna()]


,Unicode,Entidad Federativa de residencia actual_x,Género_x,Grupos quinquenales de edad_x,Población de 5 años y más migrante_x,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,...,No especificado_x,Entidad Federativa de residencia actual_y,Género_y,Grupos quinquenales de edad_y,Población de 5 años y más migrante_y,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado_y


In [82]:
df_INEGI_merged.loc[df_INEGI_merged["En otra entidad"].isna()]

,Unicode,Entidad Federativa de residencia actual_x,Género_x,Grupos quinquenales de edad_x,Población de 5 años y más migrante_x,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,...,No especificado_x,Entidad Federativa de residencia actual_y,Género_y,Grupos quinquenales de edad_y,Población de 5 años y más migrante_y,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado_y


In [84]:
df_INEGI_merged.loc[df_INEGI_merged["Entidad Federativa de residencia actual_y"].isna()]

,Unicode,Entidad Federativa de residencia actual_x,Género_x,Grupos quinquenales de edad_x,Población de 5 años y más migrante_x,Buscar trabajo,Cambio u oferta de trabajo,Reunirse con la familia,Se casó o unió,Estudiar,...,No especificado_x,Entidad Federativa de residencia actual_y,Género_y,Grupos quinquenales de edad_y,Población de 5 años y más migrante_y,En otra entidad o país,En otra entidad,En los Estados Unidos de América,En otro país,No especificado_y


### Inicio del Análsis  <a id="analisis"></a>

Verificaremos si los motivos que conllevan a la migración de México influyen en que la migración sea en los Estados Unidos de América u en otros países.




In [89]:
def separa_nombre(nombre):
    nuevo = nombre[0]
    for letra in nombre[1:]:
        if letra == letra.upper():
            nuevo += " "
        nuevo += letra
    return nuevo

ciudades_mas_pobladas = df_INEGI_merged.loc[df_INEGI_merged.Población de 5 años y más migrante_x.nlargest(5).index, 
                                    ["Entidad Federativa de residencia actual_x","Población de 5 años y más migrante_x"]]
ciudades = [ separa_nombre(Entidad Federativa de residencia actual_x) for Entidad Federativa de residencia actual_x in ciudades_mas_pobladas["Entidad Federativa de residencia actual_x"].to_list()]
colores = ["blue","gray","red","olive","brown"]
plt.figure(figsize = (10,5))
plt.bar(x= ciudades, height= ciudades_mas_pobladas["Población de 5 años y más migrante_x"]/1000, color = colores, alpha = 0.5)
plt.ylim([0,5000])
plt.ylabel("Habitantes (miles)")
plt.xlabel("Ciudades")

ciudades_mas_pobladas.set_index("Entidad Federativa de residencia actual_x", inplace = True)
for indice, ciudad in enumerate(ciudades_mas_pobladas.index):
    poblacion = ciudades_mas_pobladas.loc[ciudad,"Población de 5 años y más migrante_x"]/1000
    plt.text(ciudades[indice], poblacion + 50, f"{poblacion}", ha = "center")
plt.title("Ciudades más pobladas de California")
plt.show()


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3478201353.py, line 9)